In [1]:
!pip install gensim
!pip install scipy==1.10.1
!pip install hydra-core
!pip install vaex
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118



You should consider upgrading via the 'C:\Users\Administrator\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Administrator\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Administrator\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
ERROR: Invalid requirement: '#'
You should consider upgrading via the 'C:\Users\Administrator\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Administrator\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://download.pytorch.org/whl/cu118


You should consider upgrading via the 'C:\Users\Administrator\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import logging
import math
import os
import hydra
import json
import random
from rich.pretty import pprint
from collections import defaultdict

import pyarrow as pa
import pyarrow.feather
import vaex
import torch
from typing import Callable, Iterable, Optional, Sequence
from pathlib import Path

import os
from dataclasses import dataclass
from functools import reduce
from typing import Any, Callable, Optional

import numpy as np
import pyarrow.compute as pc
from rich.progress import track

import gensim.models.word2vec as w2v
from omegaconf import OmegaConf
from transformers import AutoTokenizer

from copy import deepcopy
from sklearn.metrics import auc, average_precision_score, roc_curve

import torch.utils.checkpoint
from torch import nn

from transformers import RobertaModel, AutoConfig

import gc
from collections import defaultdict
from pathlib import Path
from tempfile import mkdtemp
from typing import Optional

import pandas as pd
from torch.utils.data import DataLoader

## Import Librarires

## Define Model and Training Parameters

In [3]:
epochs = 1
batch_size = 1
max_batch_size = 1

model_name = "SciBERT"    # "RoBERTa" or "MedBERT"  or "SciBERT"  or "PubMedBERT"

train_split = 85595
val_split = 12227
test_split = 24456


## Definen Required Column Names and Special Tokens

In [4]:
PAD_TOKEN = "<PAD>"
UNKNOWN_TOKEN = "<UNK>"


ID_COLUMN = "_id"
TEXT_COLUMN = "text"
TARGET_COLUMN = "target"
SUBJECT_ID_COLUMN = "subject_id"

## Define the directories

In [5]:
pretrained_model_directory = ""   # Define the SciBERT directory here
preprocessed_data_file_directory = ""  #Define the preprocessed data file directory here

## Pretrained Model Path

In [6]:
pretrained_model_path = Path(pretrained_model_directory)

## Visualize the Preprocessed Data

In [7]:
import pandas as pd
df = pd.read_feather(preprocessed_data_file_directory)
df

,note_id,subject_id,_id,note_type,note_seq,charttime,storetime,text,icd10_proc,icd10_diag,target,num_words,num_targets
0,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,name unit no admission date discharge date dat...,[],"[E78.5, F02.80, G31.83, R29.6, R44.1, Z85.46]","[E78.5, F02.80, G31.83, R29.6, R44.1, Z85.46]",1305,6
1,10000117-DS-21,10000117,22927623,DS,21,2181-11-15 00:00:00,2181-11-15 15:04:00,name unit no admission date discharge date dat...,[],"[F41.9, I34.1, K21.9, K31.819, K44.9, M81.0, R...","[F41.9, I34.1, K21.9, K31.819, K44.9, M81.0, R...",626,9
2,10000117-DS-22,10000117,27988844,DS,22,2183-09-21 00:00:00,2183-09-29 16:23:00,name unit no admission date discharge date dat...,[0QS734Z],"[E78.00, F41.9, G43.909, I34.1, K21.9, M81.0, ...","[0QS734Z, E78.00, F41.9, G43.909, I34.1, K21.9...",887,13
3,10000980-DS-24,10000980,25911675,DS,24,2191-05-24 00:00:00,2191-05-24 17:29:00,name unit no admission date discharge date dat...,[],"[D50.0, E11.8, I12.9, I25.10, I25.2, I50.23, I...","[D50.0, E11.8, I12.9, I25.10, I25.2, I50.23, I...",1722,22
4,10000980-DS-25,10000980,29659838,DS,25,2191-07-19 00:00:00,2191-07-22 09:37:00,name unit no admission date discharge date dat...,[],"[D63.1, E11.21, E78.5, I12.9, I25.10, I25.2, I...","[D63.1, E11.21, E78.5, I12.9, I25.10, I25.2, I...",1547,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122273,19999784-DS-7,19999784,26194817,DS,7,2119-07-02 00:00:00,2119-07-03 16:21:00,name unit no admission date discharge date dat...,"[07DR3ZX, 0CJS8ZZ]","[C85.99, D47.2, D64.9, D72.819, E44.0, F17.210...","[07DR3ZX, 0CJS8ZZ, C85.99, D47.2, D64.9, D72.8...",2152,12
122274,19999784-DS-8,19999784,24935234,DS,8,2119-07-12 00:00:00,2119-07-14 21:18:00,name unit no admission date discharge date dat...,[],"[D47.2, F17.210, G95.89, J38.01, M54.16, R13.1...","[D47.2, F17.210, G95.89, J38.01, M54.16, R13.1...",1284,8
122275,19999784-DS-9,19999784,23664472,DS,9,2119-08-09 00:00:00,2119-08-09 19:16:00,name unit no admission date discharge date dat...,"[00UT0JZ, 02HV33Z, 0HQ6XZZ, 3E03305]","[C83.39, D47.2, E43., G96.0, G97.82, T81.32XA,...","[00UT0JZ, 02HV33Z, 0HQ6XZZ, 3E03305, C83.39, D...",1436,14
122276,19999828-DS-6,19999828,29734428,DS,6,2147-08-04 00:00:00,2147-08-12 15:36:00,name unit no admission date discharge date dat...,"[02HV33Z, 0HBHXZZ, 0HBJXZZ, 0HR7X74, 3E0436Z]","[B96.20, D68.51, E11.9, F12.90, F41.9, I10., I...","[02HV33Z, 0HBHXZZ, 0HBJXZZ, 0HR7X74, 3E0436Z, ...",1689,27


## Split the Data into Training, Validation and Testing Data

In [8]:
def assign_splits(dataframe, train_split, val_split, test_split):
    unique_ids = dataframe['_id'].unique()
    total_ids = len(unique_ids)
    total_split = train_split + val_split + test_split

    if total_split > total_ids:
        raise ValueError("Total splits exceed the total number of unique IDs.")

    train_ids = unique_ids[:train_split]
    val_ids = unique_ids[train_split:train_split+val_split]
    test_ids = unique_ids[train_split+val_split:total_split]

    split_df = pd.DataFrame({'_id': np.concatenate([train_ids, val_ids, test_ids]),
                             'split': ['train'] * train_split + ['val'] * val_split + ['test'] * test_split})

    return split_df

split_df = assign_splits(df, train_split=train_split, val_split=val_split, test_split=test_split)
split_df.to_feather("split_data.feather")


### Visualize the Splitted Dataset

In [9]:
import pandas as pd
df = pd.read_feather("split_data.feather")
# Counting occurrences of each label
label_counts = df['split'].value_counts()

# Printing the counts
print("Train count:", label_counts.get('train', 0))
print("Val count:", label_counts.get('val', 0))
print("Val count:", label_counts.get('test', 0))


df.head

Train count: 85595
Val count: 12227
Val count: 24456


<bound method NDFrame.head of              _id  split
0       25352398  train
1       20407963  train
2       29710796  train
3       29712230  train
4       25936632  train
...          ...    ...
122273  29634509   test
122274  25083567   test
122275  28167262   test
122276  23814306   test
122277  21456551   test

[122278 rows x 2 columns]>

## Define the Preprocessed Data Filename and Split Filename

In [10]:
data_filename = preprocessed_data_file_directory
split_filename = "split_data.feather"
code_column_names = ["icd10_diag", "icd10_proc"]

# **Get the Data From Data PipeLine**

### Define the Data Class

In [11]:
@dataclass
class Lookups:
    data_info: dict[str, Any]
    code2description: Optional[dict[str, str]] = None
    code_system2code_indices: Optional[dict[str, torch.Tensor]] = None
    split2code_indices: Optional[dict[str, torch.Tensor]] = None


@dataclass
class Data:
    """Dataclass containing the dataset and the code occurrences of each code system."""

    df: pa.Table
    code_system2code_counts: dict[str, dict[str, int]]

    @property
    def train(self) -> list[pa.RecordBatch]:
        """Get the training data.

        Returns:
            list[pa.RecordBatch]: List of record batches.
        """
        batches = (
            self.df.filter(pc.field("split") == "train")
            .sort_by("num_words")
            .to_batches(max_chunksize=1)
        )
        return self.from_batches_to_list(batches)

    @property
    def val(self) -> list[pa.RecordBatch]:
        """Get the validation data.

        Returns:
            list[pa.RecordBatch]: List of record batches.
        """
        batches = (
            self.df.filter(pc.field("split") == "val")
            .sort_by("num_words")
            .to_batches(max_chunksize=1)
        )
        return self.from_batches_to_list(batches)

    @property
    def test(self) -> list[pa.RecordBatch]:
        """Get the test data.

        Returns:
            list[pa.RecordBatch]: List of record batches.
        """
        batches = (
            self.df.filter(pc.field("split") == "test")
            .sort_by("num_words")
            .to_batches(max_chunksize=1)
        )
        return self.from_batches_to_list(batches)

    def from_batches_to_list(
        self,
        batches: list[pa.RecordBatch],
    ) -> list[tuple[torch.Tensor, np.array, str, int, torch.Tensor]]:
        """Convert a list of record batches to a list of tuples

        Args:
            batches (list[pa.RecordBatch]): List of record batches.

        Returns:
            list[tuple[torch.Tensor, np.array, str, int, torch.Tensor]]: List of tuples.
        """
        examples = []
        import pdb

        for batch in track(batches, description="Creating examples"):
            token_ids = torch.from_numpy(
                batch.column("token_ids")
                .flatten()
                .to_numpy(zero_copy_only=False, writable=True)
            )
            targets = (
                batch.column(TARGET_COLUMN)
                .flatten()
                .to_numpy(zero_copy_only=False, writable=True)
            )
            id = batch.column(ID_COLUMN)[0].as_py()
            num_tokens = len(token_ids)
            attention_mask = torch.ones_like(token_ids)
            examples.append((token_ids, targets, id, num_tokens, attention_mask))
        return examples

    @property
    def get_documents(self) -> list[str]:
        """Get all the documents in the dataset.

        Returns:
            list[str]: List of documents.
        """
        return self.df.column(TEXT_COLUMN).to_pylist()

    @property
    def all_target_counts(self) -> dict[str, int]:
        """Get the number of occurrences of each code in the dataset.

        Returns:
            dict[str, int]: Dictionary with the number of occurrences of each code.
        """
        return reduce(lambda x, y: {**x, **y}, self.code_system2code_counts.values())

    @property
    def get_train_documents(self) -> list[str]:
        """Get the training documents."""
        return (
            self.df.filter(pc.field("split") == "train").column(TEXT_COLUMN).to_pylist()
        )

    def split_targets(self, name: str) -> set[str]:
        """Get the targets of a split."""
        return set(
            self.df.filter(pc.field("split") == name)
            .column(TARGET_COLUMN)
            .combine_chunks()
            .flatten()
            .unique()
            .to_pylist()
        )

    def split_size(self, name: str) -> int:
        """Get the size of a split."""
        return len(self.df.filter(pc.field("split") == name))

    def num_split_targets(self, name: str) -> int:
        """Get the number of targets of a split."""
        return len(self.split_targets(name))

    @property
    def all_targets(self) -> set[str]:
        """Get all the targets in the dataset.

        Returns:
            set[str]: Set of all targets.
        """
        all_codes = set()
        for codesystem in self.code_system2code_counts.values():
            all_codes |= set(codesystem.keys())
        return all_codes

    @property
    def info(self) -> dict[str, int]:
        """Get information about the dataset.

        Returns:
            dict[str, int]: Dictionary with information about the dataset.
        """
        return {
            "num_classes": len(self.all_targets),
            "num_examples": len(self.df),
            "num_train_tokens": sum(
                self.df.filter(pc.field("split") == "train")
                .column("num_words")
                .to_pylist()
            ),
            "average_tokens_per_example": sum(self.df.column("num_words").to_pylist())
            / len(self.df),
            "num_train_examples": self.split_size("train"),
            "num_val_examples": self.split_size("val"),
            "num_test_examples": self.split_size("test"),
            "num_train_classes": self.num_split_targets("train"),
            "num_val_classes": self.num_split_targets("val"),
            "num_test_classes": self.num_split_targets("test"),
            "average_classes_per_example": sum(
                [
                    sum(codesystem.values())
                    for codesystem in self.code_system2code_counts.values()
                ]
            )
            / len(self.df),
        }

    def truncate_text(self, max_length: int) -> None:
        """Truncate text to a maximum length.

        Args:
            max_length (int): Maximum length of text.
        """
        if max_length is None:
            return

        text = self.df.column(TEXT_COLUMN)
        text_split = pc.utf8_split_whitespace(text)  # pylint: disable=no-member
        text_split_df = text_split.to_pandas().apply(lambda x: x[:max_length])
        text_split_truncate = pa.array(text_split_df.values)
        text_truncate = pc.binary_join(  # pylint: disable=no-member
            text_split_truncate, " "
        )

        # Change column in table
        new_table_no_text = self.df.drop([TEXT_COLUMN])
        new_table = new_table_no_text.append_column(
            pa.field(TEXT_COLUMN, pa.string()), text_truncate
        )
        del self.df
        self.df = new_table

    def transform_text(self, batch_transform: Callable[[list[str]], str]) -> None:
        """Transform the text using a batch transform function.

        Args:
            batch_transform (Callable[[list[str]], str]): Batch transform function.
        """
        token_ids_list = []
        for batch in track(
            self.df.to_batches(max_chunksize=10000), description="Transforming text..."
        ):
            texts = batch.column(TEXT_COLUMN).to_pylist()
            token_ids_list += batch_transform(texts)

        # Convert to list of pyarrays
        token_ids = pa.array(token_ids_list, type=pa.list_(pa.int64()))
        del token_ids_list
        # Append to table
        new_table_no_text = self.df.drop([TEXT_COLUMN])
        del self.df
        new_table = new_table_no_text.append_column(
            pa.field("token_ids", pa.list_(pa.int64())), [token_ids]
        )
        self.df = new_table
        os.environ["TOKENIZERS_PARALLELISM"] = "False"


@dataclass
class Batch:
    """Batch class. Used to store a batch of data."""

    data: torch.Tensor
    targets: torch.Tensor
    ids: torch.Tensor
    code_descriptions: Optional[torch.Tensor] = None
    num_tokens: Optional[torch.Tensor] = None
    attention_mask: Optional[torch.Tensor] = None

    def to(self, device: Any) -> "Batch":
        """Move the batch to a device.

        Args:
            device (Any): Device to move the batch to.

        Returns:
            self: Moved batch.
        """
        self.data = self.data.to(device, non_blocking=True)
        self.targets = self.targets.to(device, non_blocking=True)
        if self.attention_mask is not None:
            self.attention_mask = self.attention_mask.to(device, non_blocking=True)
        return self

    # custom memory pinning method on custom type
    def pin_memory(self):
        self.data = self.data.pin_memory()
        self.targets = self.targets.pin_memory()
        if self.attention_mask is not None:
            self.attention_mask = self.attention_mask.pin_memory()
        return self

## Data Pipeline Utils

In [12]:
def get_code_system2code_counts(
    df: vaex.dataframe.DataFrame, code_systems: list[str]
) -> dict[str, dict[str, int]]:
    """

    Args:
        df (vaex.dataframe.DataFrame): The dataset in vaex dataframe format
        code_systems (list[str]): list of code systems to get counts for
    Returns:
        dict[str, dict[str, int]]: A dictionary with code systems as keys and a dictionary of code counts as values
    """
    code_system2code_counts = defaultdict(dict)
    for col in code_systems:
        codes = df[col].values.flatten().value_counts().to_pylist()
        code_system2code_counts[col] = {
            code["values"]: code["counts"] for code in codes
        }
    return code_system2code_counts


def data_pipeline(data_file_name,
                 code_column_name,
                 split_file_name):

    with vaex.cache.memory_infinite():  # type: ignore
        df = vaex.from_arrow_table(
            pyarrow.feather.read_table(
                data_filename,
                columns=[
                    ID_COLUMN,
                    TEXT_COLUMN,
                    TARGET_COLUMN,
                    "num_words",
                    "num_targets",
                ]
                + code_column_names,
            )
        )
        splits = vaex.from_arrow_table(
            pyarrow.feather.read_table(
                split_filename,
            )
        )
        df = df.join(splits, on=ID_COLUMN, how="inner")
        code_system2code_counts = get_code_system2code_counts(
            df, code_column_names
        )
        schema = pa.schema(
            [
                pa.field(ID_COLUMN, pa.int64()),
                pa.field(TEXT_COLUMN, pa.large_utf8()),
                pa.field(TARGET_COLUMN, pa.list_(pa.large_string())),
                pa.field("split", pa.large_string()),
                pa.field("num_words", pa.int64()),
                pa.field("num_targets", pa.int64()),
            ]
        )

        return Data(
            df[
                [
                    ID_COLUMN,
                    TEXT_COLUMN,
                    TARGET_COLUMN,
                    "split",
                    "num_words",
                    "num_targets",
                ]
            ]
            .to_arrow_table()
            .cast(schema),
            code_system2code_counts,
        )

## Select and Print the Device(GPU or CPU) for Model Training

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pprint(f"Device: {device}")

'Device: cuda'

## Get Data from Data Pipeline

In [14]:
data = data_pipeline(data_file_name = data_filename,
                 code_column_name = code_column_names,
                 split_file_name = split_filename)

# **Text Encoder**

## Text Encoder Utils

In [15]:
class BaseTextEncoder:
    """The base class for text encoders."""

    def __init__(self, config):
        self.config = config

    def save(self, path: Path) -> None:
        """Save the text encoder.

        Args:
            path (Path): The path to save the text encoder to.
        """
        raise NotImplementedError

    @classmethod
    def load(cls, path: Path) -> "BaseTextEncoder":
        """Load the text encoder.

        Args:
            path (Path): The path to load the text encoder from.

        Returns:
            TextEncoder: The text encoder.
        """
        raise NotImplementedError

    def fit(self, texts: Iterable[str]) -> None:
        """Fit the text encoder.

        Args:
            texts (Iterable[str]): The texts to fit the text encoder to.
        """
        raise NotImplementedError

    @property
    def token2index(self) -> dict:
        """The token to index mapping.

        Returns:
            dict: The token to index mapping.
        """
        raise NotImplementedError

    @property
    def index2token(self) -> dict:
        """The index to token mapping.

        Returns:
            dict: The index to token mapping.
        """
        raise NotImplementedError

    @property
    def weights(self) -> np.ndarray:
        """The weights of the text encoder.

        Returns:
            np.ndarray: The weights of the text encoder.
        """
        raise NotImplementedError
def word_tokenizer(string):
    """
    Splits a string by whitespace characters.
    Args:
        string (string): The string to be split by whitespace characters.
    Returns:
        list: The words of the string.
    """
    return string.split()

class Word2Vec(BaseTextEncoder):
    """The word2vec text encoder."""

    def __init__(self, config, model: Optional[w2v.Word2Vec] = None) -> None:
        super().__init__(config)
        self.model = model
        self.tokeniser = word_tokenizer

    def save(self, path: Path) -> None:
        """Save the word2vec model.

        Args:
            path (Path): The path to save the word2vec model to.
        """
        self.model.save(str(path))
        OmegaConf.save(config=self.config, f=path.with_suffix(".yaml"))

    @classmethod
    def load(cls, path: Path) -> "Word2Vec":
        """Load the word2vec model.

        Args:
            path (Path): The path to load the word2vec model from.

        Returns:
            Word2Vec: The word2vec model.
        """
        model = w2v.Word2Vec.load(str(path))
        config = OmegaConf.load(path.with_suffix(".yaml"))
        return cls(config, model=model)

    @property
    def weights(self) -> np.ndarray:
        return self.model.wv.vectors

    @property
    def token2index(self) -> dict:
        """The token to index mapping.

        Returns:
            dict: The token to index mapping.
        """
        return self.model.wv.key_to_index

    @property
    def index2token(self) -> dict:
        """The index to token mapping.

        Returns:
            dict: The index to token mapping.
        """
        return self.model.wv.index_to_key

    @property
    def embedding_size(self) -> int:
        """The embedding size.

        Returns:
            int: The embedding size.
        """
        return self.model.vector_size

    def fit(self, texts: Iterable[str], model_configs) -> None:
        """Fit the word2vec model.

        Args:
            texts (Iterable[str]): The texts to fit the word2vec model to.
        """
        sentences = [self.tokeniser(sentence) for sentence in texts]
        self.model = w2v.Word2Vec(sentences, **model_configs)
        self.remove_rare_words(texts, 3)
        vec = np.random.randn(self.embedding_size)
        self.model.wv.add_vector(UNKNOWN_TOKEN, vec)
        self.normalize_weights()
        self.model.wv.add_vector(PAD_TOKEN, np.zeros(self.embedding_size))

    def normalize_weights(self) -> None:
        """Normalize the word2vec model."""
        self.model.init_sims(replace=True)

    def remove_rare_words(self, texts: Iterable[str], min_document_count: int) -> None:
        """Remove the rare words from the word2vec model.

        Args:
            texts (Iterable[str]): The data to train on. Each element is a sentence.
            min_document_count (int): The minimum number of documents a word occurs in.
        """

        word_counts = dict()
        for sentence in texts:
            words_in_sentence = set()
            for word in sentence.split():
                if word not in word_counts:
                    word_counts[word] = 0
                if word not in words_in_sentence:
                    word_counts[word] += 1
                words_in_sentence.add(word)

        words_to_remove = [
            word
            for word, count in word_counts.items()
            if (count < min_document_count) and (word in self.model.wv.key_to_index)
        ]
        ids_to_remove = [self.model.wv.key_to_index[word] for word in words_to_remove]

        for word in words_to_remove:
            del self.model.wv.key_to_index[word]

        self.model.wv.vectors = np.delete(self.model.wv.vectors, ids_to_remove, axis=0)

        for i in sorted(ids_to_remove, reverse=True):
            del self.model.wv.index_to_key[i]

        for i, key in enumerate(self.model.wv.index_to_key):
            self.model.wv.key_to_index[key] = i

In [16]:
text_encoder_configs = {
    "min_document_count": 3
}
model_configs = {
        "vector_size": 100,
        "min_count": 0,
        "workers": 4,
        "epochs": 5
    }

In [17]:
def get_text_encoder( texts):
    path = Path("word2vec_full.model")
    text_encoder = Word2Vec(config=text_encoder_configs)
    text_encoder.fit(texts, model_configs)
    text_encoder.save(path)
    return text_encoder

## Get the Text Encoder

In [ ]:
text_encoder = get_text_encoder(texts=data.get_train_documents)

# **Get Label and Text Transforms**

### Transform Utils

In [ ]:
class Transform(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.unknown_token = UNKNOWN_TOKEN
        self.pad_token = PAD_TOKEN
        self.pad_index = 0
        self.unknown_index = None

    def forward(self, x: Any):
        return self.transform(x)

    def transform(self, x):
        raise NotImplementedError()

    def inverse_transform(self, x):
        raise NotImplementedError()

    def seq2batch(self, sequence) -> torch.Tensor:
        """A batching function for the Transform classes. Used in the collate function of the dataset.

        Args:
            sequence (Sequence[torch.Tensor]): A Sequence of vectors of equal lengths.

        Returns:
            torch.Tensor: A tensor of vectors.
        """
        return torch.stack(sequence)

    def batch_transform(self, texts: list[str]) -> list[list[int]]:
        """Transform a batch of texts into a batch of indices.

        Args:
            texts (list[str]): A batch of texts.

        Returns:
            torch.Tensor: A batch of indices.
        """
        raise NotImplementedError

class OneHotEncoder(Transform):
    def __init__(self) -> None:
        """One hot encoder for targets"""
        super().__init__()
        self.target2index = {}
        self.index2target = {}
        self.file_name = "target2index.json"

    def fit(self, targets: set[str]) -> None:
        """Fit the encoder to all the targets in the dataset. That also includes the validation and test set.

        Args:
            targets (set[str]): List of targets
        """
        for index, target in enumerate(targets):
            self.target2index[target] = index
            self.index2target[index] = target

    @property
    def num_classes(self) -> int:
        """Number of classes supported by the encoder

        Returns:
            int: Number of classes
        """
        return len(self.target2index)

    def get_classes(self) -> list[str]:
        """Get the list of classes supported by the encoder. The classes are sorted by their index."""
        return [self.index2target[index] for index in range(self.num_classes)]

    def transform(self, targets: Iterable[str]) -> torch.Tensor:
        """Transform a set of targets into a one-hot encoded tensor

        Args:
            targets (set[str]): Set of targets

        Returns:
            torch.Tensor: One-hot encoded tensor
        """
        output_tensor = torch.zeros(self.num_classes)
        for label in targets:
            if label in self.target2index:
                output_tensor[self.target2index[label]] = 1
        return output_tensor

    def inverse_transform(self, output_tensor: torch.Tensor) -> set[str]:
        """Transform a one-hot encoded tensor into a set of targets

        Args:
            output_tensor (torch.Tensor): One-hot encoded tensor

        Returns:
            set[str]: Set of targets
        """

        indices = torch.nonzero(output_tensor).squeeze(0).numpy()
        return set([self.index2target[int(index)] for index in indices])

    def get_indices(self, targets: Iterable[str]) -> torch.Tensor:
        """Get the indices of the targets

        Args:
            targets (Iterable[str]): Set of targets

        Returns:
            torch.Tensor: Indices of the targets
        """
        indices = torch.zeros(len(targets), dtype=torch.long)
        for index, label in enumerate(targets):
            if label in self.target2index:
                indices[index] = self.target2index[label]
        return indices

    def save(self, path: str) -> None:
        """Save target2index as a json file

        Args:
            path (str): path to save the json file
        """
        path = Path(path) / self.file_name
        with open(path, "w") as f:
            json.dump(self.target2index, f)

    def load(self, path: str) -> None:
        """Load target2index from a json file

        Args:
            path (str): path of the directory containing the json file
        """
        path = Path(path) / self.file_name
        with open(path, "r") as f:
            self.target2index = json.load(f)
            self.index2target = {v: k for k, v in self.target2index.items()}


class HuggingFaceTokenizer(Transform):
    def __init__(
        self,
        model_path: str,
        add_special_token: bool = True,
        padding: bool = False,
        truncation: bool = False,
        max_length: Optional[int] = None,
        **kwargs
    ):
        """A class to encode text into indices and back.

        Args:
            model_name (str): The name of the huggingface tokenizer.
        """
        super().__init__()
        self.add_special_token = add_special_token
        self.padding = padding
        self.max_length = max_length
        self.truncation = truncation
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, **kwargs)

    def transform(self, text: str) -> torch.Tensor:
        """Transform a text into a list of indices.

        Args:
            text (str): The text to be transformed.

        Returns:
            torch.Tensor: A tensor of indices.
        """
        return self.tokenizer(
            text,
            padding=self.padding,
            truncation=self.truncation,
            add_special_tokens=self.add_special_token,
            max_length=self.max_length,
            return_tensors="pt",
        )

    def batch_transform(self, texts: list[str]) -> list[list[int]]:
        """Transform a batch of texts into a batch of indices.

        Args:
            texts (list[str]): A batch of texts.

        Returns:
            list[list[int]]: A batch of indices.
        """
        return self.tokenizer(
            texts,
            padding=self.padding,
            truncation=self.truncation,
            add_special_tokens=self.add_special_token,
            max_length=self.max_length,
        )["input_ids"]

    def inverse_transform(self, indices: list[str]) -> list[str]:
        """Transform a list of indices into a list of tokens.

        Args:
            indices (torch.Tensor): A tensor of indices.

        Returns:
            list[str]: A list of tokens.
        """

        return self.tokenizer.decode(indices)

    def seq2batch(
        self, sequence: Sequence[torch.Tensor], chunk_size: int = 0
    ) -> torch.Tensor:
        """Batch a sequence of vectors of different lengths. Use the pad_index to pad the vectors.

        Args:
            sequences (Sequence[torch.Tensor]): A sequence of sequences.

        Returns:
            torch.Tensor: A batched tensor.
        """
        if chunk_size == 0:
            return torch.nn.utils.rnn.pad_sequence(
                sequence, batch_first=True, padding_value=self.tokenizer.pad_token_id
            )
        sequence = list(sequence)
        batch_size = len(sequence)
        max_length = max([len(x) for x in sequence])
        if max_length % chunk_size != 0:
            max_length = max_length + (chunk_size - max_length % chunk_size)

        # pad first sequence to the desired length
        sequence[0] = torch.nn.functional.pad(
            sequence[0],
            (0, max_length - len(sequence[0])),
            value=self.tokenizer.pad_token_id,
        )
        return (
            torch.nn.utils.rnn.pad_sequence(
                sequence, batch_first=True, padding_value=self.tokenizer.pad_token_id
            )
            .contiguous()
            .view((batch_size, -1, chunk_size))
        )

    @property
    def vocab_size(self) -> int:
        """The size of the vocabulary.

        Returns:
            int: The size of the vocabulary.
        """
        return len(self.tokenizer)

    def fit(self, texts: list[str]) -> None:
        pass

    def save(self, path: str) -> None:
        pass

    def load(self, path: str) -> None:
        pass

## Transform Methods

In [ ]:
def get_transform_targets(
    targets: Optional[set[str]] = data.all_targets,
    text_encoder: Optional[BaseTextEncoder] = None):

    transform_class = OneHotEncoder()

    if text_encoder:
        transform_class.set_tokenmap(
            token2index=text_encoder.token2index, index2token=text_encoder.index2token
        )


    elif targets:
        transform_class.fit(targets)

    else:
        raise ValueError(
            "Provide set of labels, a text encoder or texts of tokens to perform fit transformation"
        )

    return transform_class

def get_transform_texts(
    targets: Optional[set[str]] = None,
    texts: Optional[list[str]] = None,
    text_encoder: Optional[BaseTextEncoder] = None,
    load_transform_path: Optional[str] = None,):

    transform_class = HuggingFaceTokenizer(model_path = pretrained_model_path ,
                                          add_special_token = True,
                                          padding = False,
                                          max_length = None,
                                           use_fast = True,
                                           do_lower_case = True,
                                           truncation = False
                                          )
    if load_transform_path:
        transform_class.load(load_transform_path)
        print("loaded transform")

    elif text_encoder:
        transform_class.set_tokenmap(
            token2index=text_encoder.token2index, index2token=text_encoder.index2token
        )

    elif texts:
        transform_class.fit(texts)

    elif targets:
        transform_class.fit(targets)

    else:
        raise ValueError(
            "Provide set of labels, a text encoder or texts of tokens to perform fit transformation"
        )

    return transform_class

## Get Label and Text Transforms

In [ ]:
label_transform = get_transform_targets()
text_transform = get_transform_texts(
        texts=data.get_train_documents,
        text_encoder=text_encoder,
        load_transform_path=True,
    )

## Transform the Data

In [ ]:
data.truncate_text(4000)
data.transform_text(text_transform.batch_transform)

## Lookup Utils

In [ ]:
def get_data_info(data: Data, vocab_size: int, pad_index: int) -> dict:
    data_info = data.info
    data_info["vocab_size"] = vocab_size
    data_info["pad_index"] = pad_index
    return data_info

def get_code_system2code_indices(
    data: Data, label_transform: Transform
) -> dict[str, torch.Tensor]:
    code_system2code_indices = {}
    for codesystem, codes in data.code_system2code_counts.items():
        code_system2code_indices[codesystem] = label_transform.get_indices(
            set(codes.keys())
        )
    return code_system2code_indices


def get_split2code_indices(
    data: Data, label_transform: Transform
) -> dict[str, torch.Tensor]:
    split2code_indices = {}
    split2code_indices["train"] = label_transform.get_indices(
        data.split_targets("train")
    )
    split2code_indices["train_val"] = label_transform.get_indices(
        data.split_targets("train")
    )
    split2code_indices["val"] = label_transform.get_indices(data.split_targets("val"))
    split2code_indices["test"] = label_transform.get_indices(data.split_targets("test"))
    return split2code_indices


def load_lookups(
    data: Data,
    label_transform: Transform,
    text_transform: Transform,
) -> Lookups:
    """Load the lookups.

    Args:
        config (OmegaConf): The config.

    Returns:
        Lookups: The lookups.
    """
    data_info = get_data_info(
        data, text_transform.vocab_size, label_transform.pad_index
    )
    code_system2code_indices = get_code_system2code_indices(data, label_transform)
    split2code_indices = get_split2code_indices(data, label_transform)

    return Lookups(
        data_info=data_info,
        code_system2code_indices=code_system2code_indices,
        split2code_indices=split2code_indices,
    )

def get_lookups(
    data: Data,
    label_transform,
    text_transform,
) -> Lookups:
    return load_lookups(
        data=data,
        label_transform=label_transform,
        text_transform=text_transform,
    )


## Get Lookups

In [ ]:
lookups = get_lookups(
        data=data,
        label_transform=label_transform,
        text_transform=text_transform,
    )


## **Define the Model**

In [ ]:
from transformers import AutoTokenizer, BertModel
class LabelAttention(nn.Module):
    def __init__(self, input_size: int, projection_size: int, num_classes: int):
        super().__init__()
        self.first_linear = nn.Linear(input_size, projection_size, bias=False)
        self.second_linear = nn.Linear(projection_size, num_classes, bias=False)
        self.third_linear = nn.Linear(input_size, num_classes)
        self._init_weights(mean=0.0, std=0.03)
        self.layer_norm = nn.LayerNorm(normalized_shape=num_classes)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """LAAT attention mechanism

        Args:
            x (torch.Tensor): [batch_size, seq_len, input_size]

        Returns:
            torch.Tensor: [batch_size, num_classes]
        """
        weights = torch.tanh(self.first_linear(x))
        att_weights = self.second_linear(weights)
        att_weights = self.layer_norm(att_weights)
        att_weights = torch.nn.functional.softmax(att_weights, dim=1).transpose(1, 2)
        weighted_output = att_weights @ x
        return (
            self.third_linear.weight.mul(weighted_output)
            .sum(dim=2)
            .add(self.third_linear.bias)
        )

    def _init_weights(self, mean: float = 0.0, std: float = 0.03) -> None:
        """
        Initialise the weights

        Args:
            mean (float, optional): Mean of the normal distribution. Defaults to 0.0.
            std (float, optional): Standard deviation of the normal distribution. Defaults to 0.03.
        """

        torch.nn.init.normal_(self.first_linear.weight, mean, std)
        torch.nn.init.normal_(self.second_linear.weight, mean, std)
        torch.nn.init.normal_(self.third_linear.weight, mean, std)


class PLMICD(nn.Module):
    def __init__(self, num_classes: int, model_path: str, **kwargs):
        super().__init__()
        self.config = AutoConfig.from_pretrained(
            model_path, num_labels=num_classes, finetuning_task=None
        )
        self.model = BertModel.from_pretrained(model_path, config=self.config)
        self.attention = LabelAttention(
            input_size=self.config.hidden_size,
            projection_size=self.config.hidden_size,
            num_classes=num_classes,
        )
        self.loss = torch.nn.functional.binary_cross_entropy_with_logits

    def get_loss(self, logits, targets):
        return self.loss(logits, targets)

    def training_step(self, batch) -> dict[str, torch.Tensor]:
        data, targets, attention_mask = batch.data, batch.targets, batch.attention_mask
        logits = self(data, attention_mask)
        loss = self.get_loss(logits, targets)
        logits = torch.sigmoid(logits)
        return {"logits": logits, "loss": loss, "targets": targets}

    def validation_step(self, batch) -> dict[str, torch.Tensor]:
        data, targets, attention_mask = batch.data, batch.targets, batch.attention_mask
        logits = self(data, attention_mask)
        loss = self.get_loss(logits, targets)
        logits = torch.sigmoid(logits)
        return {"logits": logits, "loss": loss, "targets": targets}

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
    ):
        r"""
        input_ids (torch.LongTensor of shape (batch_size, num_chunks, chunk_size))
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, num_labels)`, `optional`):
        """

        batch_size, num_chunks, chunk_size = input_ids.size()
        outputs = self.model(
            input_ids.view(-1, chunk_size),
            attention_mask=attention_mask.view(-1, chunk_size)
            if attention_mask is not None
            else None,
            return_dict=False,
        )

        hidden_output = outputs[0].view(batch_size, num_chunks * chunk_size, -1)
        logits = self.attention(hidden_output)
        return logits

In [ ]:
def get_model(data_info: dict, text_encoder: Optional[Any] = None):
    model_class = PLMICD(model_path = pretrained_model_path , text_encoder = text_encoder, **data_info)
    return model_class

## Get the Model

In [ ]:
model = get_model(data_info=lookups.data_info, text_encoder=text_encoder)

In [ ]:
model.to(device)

# print data info
pprint(lookups.data_info)

## Metric Utils

In [ ]:
def detach(x):
    """Detach a tensor from the computational graph"""
    if isinstance(x, torch.Tensor):
        return x.detach()
    return x


class Metric:
    base_tags = set()
    _str_value_fmt = "<.3"
    higher_is_better = True
    batch_update = True
    filter_codes = True

    def __init__(
        self,
        name: str,
        tags: set,
        number_of_classes: int,
        threshold: Optional[float] = None,
    ):
        self.name = name
        self.tags = self.base_tags if tags is None else (tags | self.base_tags)
        self.number_of_classes = number_of_classes
        self.device = "cpu"
        self.threshold = threshold
        self.reset()

    def update(self, batch: dict):
        """Update the metric from a batch"""
        raise NotImplementedError()

    def set_target_boolean_indices(self, target_boolean_indices: list[bool]):
        self.target_boolean_indices = target_boolean_indices

    def compute(self):
        """Compute the metric value"""
        raise NotImplementedError()

    def reset(self):
        """Reset the metric"""
        raise NotImplementedError()

    def to(self, device: str):
        self.device = device
        if self.threshold is not None:
            self.threshold = torch.tensor(self.threshold).clone().to(device)
        self.reset()
        return self

    def copy(self):
        return deepcopy(self)

    def set_number_of_classes(self, number_of_classes: int):
        self.number_of_classes = number_of_classes


class MetricCollection:
    def __init__(
        self,
        metrics: list[Metric],
        code_indices: Optional[torch.Tensor] = None,
        code_system_name: Optional[str] = None,
    ):
        self.metrics = metrics
        self.code_system_name = code_system_name
        if code_indices is not None:
            # Get overlapping indices
            self.code_indices = code_indices.clone()
            self.set_number_of_classes(len(code_indices))
        else:
            self.code_indices = None
        self.reset()

    def set_number_of_classes(self, number_of_classes_split: int):
        """Sets the number of classes for metrics with the filter_codes attribute to the number of classes in the split.
        Args:
            number_of_classes_split (int): Number of classes in the split
        """
        for metric in self.metrics:
            if metric.filter_codes:
                metric.set_number_of_classes(number_of_classes_split)

    def to(self, device: str):
        self.metrics = [metric.to(device) for metric in self.metrics]
        if self.code_indices is not None:
            self.code_indices = self.code_indices.to(device)
        return self

    def filter_batch(self, batch: dict) -> dict:
        if self.code_indices is None:
            return batch

        filter_batch = {}
        targets, logits = batch["targets"], batch["logits"]

        filtered_targets = torch.index_select(targets, -1, self.code_indices)
        filtered_logits = torch.index_select(logits, -1, self.code_indices)
        # Elements in the batch with targets
        idx_targets = torch.sum(filtered_targets, dim=-1) > 0
        # Remove all elements wihtout targets
        filter_batch["targets"] = filtered_targets[idx_targets]
        filter_batch["logits"] = filtered_logits[idx_targets]
        return filter_batch

    def filter_tensor(
        self, tensor: torch.Tensor, code_indices: torch.Tensor
    ) -> list[torch.Tensor]:
        if code_indices is None:
            return tensor
        return torch.index_select(tensor, -1, code_indices)

    def is_best(
        self,
        prev_best: Optional[torch.Tensor],
        current: torch.Tensor,
        higher_is_better: bool,
    ) -> bool:
        if higher_is_better:
            return prev_best is None or current > prev_best
        else:
            return prev_best is None or current < prev_best

    def update_best_metrics(self, metric_dict: dict[str, torch.Tensor]):
        for metric in self.metrics:
            if metric.name not in metric_dict:
                continue

            if self.is_best(
                self.best_metrics[metric.name],
                metric_dict[metric.name],
                metric.higher_is_better,
            ):
                self.best_metrics[metric.name] = metric_dict[metric.name]

    def update(self, batch: dict):
        for metric in self.metrics:
            if metric.batch_update and not metric.filter_codes:
                metric.update(batch)

        filtered_batch = self.filter_batch(batch)

        for metric in self.metrics:
            if metric.batch_update and metric.filter_codes:
                metric.update(filtered_batch)

    def compute(
        self,
        logits: Optional[torch.Tensor] = None,
        targets: Optional[torch.Tensor] = None,
    ) -> dict[str, torch.Tensor]:
        metric_dict = {
            metric.name: metric.compute()
            for metric in self.metrics
            if metric.batch_update
        }
        if logits is not None and targets is not None:
            # Compute the metrics for the whole dataset
            if self.code_indices is not None:
                logits_filtered = self.filter_tensor(logits, self.code_indices.cpu())
                targets_filtered = self.filter_tensor(targets, self.code_indices.cpu())

            for metric in self.metrics:
                if metric.batch_update:
                    continue
                if metric.filter_codes and self.code_indices is not None:
                    metric_dict[metric.name] = metric.compute(
                        logits=logits_filtered, targets=targets_filtered
                    )
                else:
                    metric_dict[metric.name] = metric.compute(
                        logits=logits, targets=targets
                    )

            metric_dict.update(
                {
                    metric.name: metric.compute(logits=logits, targets=targets)
                    for metric in self.metrics
                    if not metric.batch_update
                }
            )
        self.update_best_metrics(metric_dict)
        return metric_dict

    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset()

    def reset(self):
        self.reset_metrics()
        self.best_metrics = {metric.name: None for metric in self.metrics}

    def get_best_metric(self, metric_name: str) -> dict[str, torch.Tensor]:
        return self.best_metrics[metric_name]

    def copy(self):
        return deepcopy(self)

    def set_threshold(self, threshold: float):
        for metric in self.metrics:
            if hasattr(metric, "threshold"):
                metric.threshold = threshold


""" ------------Classification Metrics-------------"""


class ExactMatchRatio(Metric):
    _str_value_fmt = "6.4"  # 6.4321

    def __init__(
        self,
        threshold: float = 0.5,
        name: str = "exact_match_ratio",
        tags: set[str] = None,
        number_of_classes: int = 0,
        filter_codes: bool = True,
    ):
        if not filter_codes:
            name = f"{name}_mullenbach"
        super().__init__(
            name=name,
            tags=tags,
            number_of_classes=number_of_classes,
            threshold=threshold,
        )
        self.filter_codes = filter_codes

    def update(self, batch: dict):
        logits, targets = detach(batch["logits"]), detach(batch["targets"])
        predictions = (logits > self.threshold).long()
        self._num_exact_matches += torch.all(
            torch.eq(predictions, targets), dim=-1
        ).sum()
        self._num_examples += targets.size(0)

    def compute(self) -> torch.Tensor:
        return self._num_exact_matches / self._num_examples

    def reset(self):
        self._num_exact_matches = torch.tensor(0).to(self.device)
        self._num_examples = 0


class Recall(Metric):
    _str_value_fmt = "6.4"  # 6.4321

    def __init__(
        self,
        number_of_classes: int,
        threshold: float = 0.5,
        average: str = "micro",
        name: str = "recall",
        tags: set[str] = None,
        filter_codes: bool = True,
    ):
        if average:
            name = f"{name}_{average}"
        if not filter_codes:
            name = f"{name}_mullenbach"
        super().__init__(
            name=name,
            tags=tags,
            number_of_classes=number_of_classes,
            threshold=threshold,
        )
        self._average = average
        self.filter_codes = filter_codes

    def update(self, batch: dict):
        logits, targets = detach(batch["logits"]), detach(batch["targets"])
        predictions = (logits > self.threshold).long()
        self._tp += torch.sum(predictions * targets, dim=0)
        self._fn += torch.sum((1 - predictions) * targets, dim=0)

    def compute(self) -> torch.Tensor:
        if self._average == "micro":
            return (self._tp.sum() / (self._tp.sum() + self._fn.sum() + 1e-10)).cpu()
        if self._average == "macro":
            return torch.mean(self._tp / (self._tp + self._fn + 1e-10)).cpu()
        if self._average is None or self._average == "none":
            return (self._tp / (self._tp + self._fn + 1e-10)).cpu()
        raise ValueError(f"Invalid average: {self._average}")

    def reset(self):
        self._tp = torch.zeros((self.number_of_classes)).to(self.device)
        self._fn = torch.zeros((self.number_of_classes)).to(self.device)


class Precision(Metric):
    _str_value_fmt = "6.4"  # 6.4321

    def __init__(
        self,
        number_of_classes: int,
        threshold: float = 0.5,
        average: str = "micro",
        name: str = "precision",
        tags: set[str] = None,
        filter_codes: bool = True,
    ):
        if average:
            name = f"{name}_{average}"
        if not filter_codes:
            name = f"{name}_mullenbach"
        super().__init__(
            name=name,
            tags=tags,
            number_of_classes=number_of_classes,
            threshold=threshold,
        )
        self._average = average
        self.filter_codes = filter_codes

    def update(self, batch: dict):
        logits, targets = detach(batch["logits"]), detach(batch["targets"])
        predictions = (logits > self.threshold).long()
        self._tp += torch.sum(predictions * targets, dim=0)
        self._fp += torch.sum((predictions) * (1 - targets), dim=0)

    def compute(self):
        if self._average == "micro":
            return (self._tp.sum() / (self._tp.sum() + self._fp.sum() + 1e-10)).cpu()
        if self._average == "macro":
            return torch.mean(self._tp / (self._tp + self._fp + 1e-10)).cpu()
        if self._average is None or self._average == "none":
            return (self._tp / (self._tp + self._fp + 1e-10)).cpu()
        raise ValueError(f"Invalid average: {self._average}")

    def reset(self):
        self._tp = torch.zeros((self.number_of_classes)).to(self.device)
        self._fp = torch.zeros((self.number_of_classes)).to(self.device)


class FPR(Metric):
    _str_value_fmt = "6.4"  # 6.4321
    higher_is_better = False

    def __init__(
        self,
        number_of_classes: int,
        threshold: float = 0.5,
        average: str = "micro",
        name: str = "fpr",
        tags: set[str] = None,
        filter_codes: bool = True,
    ):
        if average:
            name = f"{name}_{average}"
        if not filter_codes:
            name = f"{name}_mullenbach"
        super().__init__(
            name=name,
            tags=tags,
            number_of_classes=number_of_classes,
            threshold=threshold,
        )
        self._average = average
        self.filter_codes = filter_codes

    def update(self, batch: dict):
        logits, targets = detach(batch["logits"]), detach(batch["targets"])
        predictions = (logits > self.threshold).long()
        self._fp += torch.sum(predictions * (1 - targets), dim=0)
        self._tn += torch.sum((1 - predictions) * (1 - targets), dim=0)

    def compute(self) -> torch.Tensor:
        if self._average == "micro":
            return (self._fp.sum() / (self._fp.sum() + self._tn.sum() + 1e-10)).cpu()
        if self._average == "macro":
            return torch.mean(self._fp / (self._fp + self._tn + 1e-10)).cpu()
        if self._average is None or self._average == "none":
            return (self._fp / (self._fp + self._tn + 1e-10)).cpu()
        raise ValueError(f"Invalid average: {self._average}")

    def reset(self):
        self._fp = torch.zeros((self.number_of_classes)).to(self.device)
        self._tn = torch.zeros((self.number_of_classes)).to(self.device)


class AUC(Metric):
    _str_value_fmt = "6.4"  # 6.4321
    batch_update = False

    def __init__(
        self,
        average: str = "micro",
        name: str = "auc",
        tags: set[str] = None,
        number_of_classes: Optional[int] = None,
        filter_codes: bool = True,
    ):
        """Area under the ROC curve. All classes that have no positive examples are ignored as implemented by Mullenbach et al. Please note that all the logits and targets are stored in the GPU memory if they have not already been moved to the CPU.

        Args:
            logits (torch.Tensor): logits from a machine learning model. [batch_size, num_classes]
            name (str, optional): name of the metric. Defaults to "auc".
            tags (set[str], optional): metrics tages. Defaults to None.
            log_to_console (bool, optional): whether to print this metric. Defaults to True.
            number_of_classes (int, optional): number of classes. Defaults to None.
            filter_codes (bool, optional): whether to filter out codes that have no positive examples. Defaults to True.
        """
        if average:
            name = f"{name}_{average}"
        if not filter_codes:
            name = f"{name}_mullenbach"
        super().__init__(name=name, tags=tags, number_of_classes=number_of_classes)
        self._average = average
        self.filter_codes = filter_codes

    def compute(self, logits: torch.Tensor, targets: torch.Tensor) -> np.float32:
        logits = detach(logits).numpy()
        targets = detach(targets).numpy()
        if self._average == "micro":
            fprs, tprs, _ = self.roc_curve(
                logits=logits, targets=targets, average=self._average
            )
            value = auc(fprs, tprs)
        if self._average == "macro":
            fprs, tprs, _ = self.roc_curve(
                logits=logits, targets=targets, average="none"
            )
            value = np.mean([auc(fpr, tpr) for fpr, tpr in zip(fprs, tprs)])
        return value

    def update(self, batch: dict):
        raise NotImplementedError("AUC is not batch updateable.")

    def roc_curve(
        self,
        logits: list[torch.Tensor],
        targets: list[torch.Tensor],
        average: str = "micro",
    ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        thresholds = torch.linspace(0, 1, 1000)

        if average == "micro":
            return roc_curve(targets.ravel(), logits.ravel())
        if average == "none":
            fprs, tprs, thresholds = [], [], []
            for i in range(targets.shape[1]):
                if targets[:, i].sum() == 0:
                    continue

                fpr, tpr, threshold = roc_curve(targets[:, i], logits[:, i])

                if np.any(np.isnan(fpr)) or np.any(np.isnan(tpr)):
                    continue

                fprs.append(fpr)
                tprs.append(tpr)
                thresholds.append(threshold)

            return fprs, tprs, thresholds
        raise ValueError(f"Invalid average: {average}")

    def reset(self):
        pass


class F1Score(Metric):
    _str_value_fmt = "6.4"  # 6.4321

    def __init__(
        self,
        number_of_classes: int,
        threshold: float = 0.5,
        average: str = "micro",
        name: str = "f1",
        tags: set[str] = None,
        filter_codes: bool = True,
    ):
        if average:
            name = f"{name}_{average}"
        if not filter_codes:
            name = f"{name}_mullenbach"
        super().__init__(
            name=name,
            tags=tags,
            number_of_classes=number_of_classes,
            threshold=threshold,
        )
        self._average = average
        self.filter_codes = filter_codes

    def update(self, batch: dict):
        logits, targets = detach(batch["logits"]), detach(batch["targets"])
        predictions = (logits > self.threshold).long()
        self._tp += torch.sum((predictions) * (targets), dim=0)
        self._fp += torch.sum(predictions * (1 - targets), dim=0)
        self._fn += torch.sum((1 - predictions) * targets, dim=0)

    def compute(self):
        if self._average == "micro":
            return (
                self._tp.sum()
                / (self._tp.sum() + 0.5 * (self._fp.sum() + self._fn.sum()) + 1e-10)
            ).cpu()
        if self._average == "macro":
            return torch.mean(
                self._tp / (self._tp + 0.5 * (self._fp + self._fn) + 1e-10)
            ).cpu()
        if self._average is None or self._average == "none":
            return (self._tp / (self._tp + 0.5 * (self._fp + self._fn) + 1e-10)).cpu()
        raise ValueError(f"Invalid average: {self._average}")

    def reset(self):
        self._tp = torch.zeros((self.number_of_classes)).to(self.device)
        self._fp = torch.zeros((self.number_of_classes)).to(self.device)
        self._fn = torch.zeros((self.number_of_classes)).to(self.device)


""" ------------Information Retrieval Metrics-------------"""


class PrecisionAtRecall(Metric):
    _str_value_fmt = "6.4"  # 6.4321

    def __init__(
        self,
        name: str = "precision@recall",
        tags: set[str] = None,
        number_of_classes: Optional[int] = None,
        filter_codes: bool = True,
    ):
        if not filter_codes:
            name = f"{name}_mullenbach"
        super().__init__(name=name, tags=tags, number_of_classes=number_of_classes)
        self.filter_codes = filter_codes

    def update(self, batch: dict):
        logits, targets = detach(batch["logits"]), detach(batch["targets"])
        num_targets = targets.sum(dim=1, dtype=torch.int64)
        _, indices = torch.sort(logits, dim=1, descending=True)
        sorted_targets = targets.gather(1, indices)
        sorted_targets_cum = torch.cumsum(sorted_targets, dim=1)
        self._precision_sum += torch.sum(
            sorted_targets_cum.gather(1, num_targets.unsqueeze(1) - 1).squeeze()
            / num_targets
        )
        self._num_examples += logits.size(0)

    def compute(self) -> torch.Tensor:
        return self._precision_sum.cpu() / self._num_examples

    def reset(self):
        self._num_examples = 0
        self._precision_sum = torch.tensor(0.0).to(self.device)


class Precision_K(Metric):
    _str_value_fmt = "6.4"  # 6.4321

    def __init__(
        self,
        k: int = 10,
        name: str = "precision",
        tags: set[str] = None,
        number_of_classes: Optional[int] = None,
        filter_codes: bool = True,
    ):
        name = f"{name}@{k}"
        if not filter_codes:
            name = f"{name}_mullenbach"
        super().__init__(name=name, tags=tags, number_of_classes=number_of_classes)
        self._k = k
        self.filter_codes = filter_codes

    def update(self, batch: dict):
        logits, targets = detach(batch["logits"]), detach(batch["targets"])
        top_k = torch.topk(logits, dim=1, k=self._k)

        targets_k = targets.gather(1, top_k.indices)
        logits_k = torch.ones(targets_k.shape, device=targets_k.device)

        tp = torch.sum(logits_k * targets_k, dim=1)
        fp = torch.sum((logits_k) * (1 - targets_k), dim=1)
        self._num_examples += logits.size(0)
        self._precision_sum += torch.sum(tp / (tp + fp + 1e-10))

    def compute(self) -> torch.Tensor:
        return self._precision_sum.cpu() / self._num_examples

    def reset(self):
        self._num_examples = 0
        self._precision_sum = torch.tensor(0.0).to(self.device)


class MeanAveragePrecision(Metric):
    _str_value_fmt = "6.4"  # 6.4321

    def __init__(
        self,
        name: str = "map",
        tags: set[str] = None,
        number_of_classes: Optional[int] = None,
        filter_codes: bool = True,
    ):
        if not filter_codes:
            name = f"{name}_mullenbach"
        super().__init__(name=name, tags=tags, number_of_classes=number_of_classes)
        self.filter_codes = filter_codes

    def update(self, batch: dict):
        logits, targets = detach(batch["logits"]), detach(batch["targets"])
        _, indices = torch.sort(logits, dim=1, descending=True)
        sorted_targets = targets.gather(1, indices)
        sorted_targets_cum = torch.cumsum(sorted_targets, dim=1)
        batch_size = logits.size(0)
        denom = torch.arange(1, targets.shape[1] + 1, device=targets.device).repeat(
            batch_size, 1
        )
        prec_at_k = sorted_targets_cum / denom
        average_precision_batch = torch.sum(
            prec_at_k * sorted_targets, dim=1
        ) / torch.sum(sorted_targets, dim=1)
        self._average_precision_sum += torch.sum(average_precision_batch)
        self._num_examples += batch_size

    def compute(self) -> torch.Tensor:
        return self._average_precision_sum.cpu() / self._num_examples

    def reset(self):
        self._num_examples = 0
        self._average_precision_sum = torch.tensor(0.0).to(self.device)


class Recall_K(Metric):
    _str_value_fmt = "6.4"  # 6.4321

    def __init__(
        self,
        k: int = 10,
        name: str = "recall",
        tags: set[str] = None,
        number_of_classes: Optional[int] = None,
        filter_codes: bool = True,
    ):
        name = f"{name}@{k}"
        if not filter_codes:
            name = f"{name}_mullenbach"
        super().__init__(name=name, tags=tags, number_of_classes=number_of_classes)
        self._k = k
        self.filter_codes = filter_codes

    def update(self, batch: dict):
        logits, targets = detach(batch["logits"]), detach(batch["targets"])
        top_k = torch.topk(logits, dim=1, k=self._k)

        targets_k = targets.gather(1, top_k.indices)
        logits_k = torch.ones(targets_k.shape, device=targets_k.device)

        tp = torch.sum(logits_k * targets_k, dim=1)
        total_number_of_relevant_targets = torch.sum(targets, dim=1)

        self._num_examples += logits.size(0)
        self._recall_sum += torch.sum(tp / (total_number_of_relevant_targets + 1e-10))

    def compute(self) -> torch.Tensor:
        return self._recall_sum.cpu() / self._num_examples

    def reset(self):
        self._num_examples = 0
        self._recall_sum = torch.tensor(0.0).to(self.device)


""" ------------Running Mean Metrics-------------"""


class RunningMeanMetric(Metric):
    _str_value_fmt = "<.3"

    def __init__(
        self,
        name: str,
        tags: set[str],
        number_of_classes: Optional[int] = None,
    ):
        """Create a running mean metric.

        Args:
            name (str): Name of the metric
            tags (Set[str]): Tags to use for grouping with other metrics.
            number_of_classes (Optional[int], optional): Number of classes. Defaults to None.
        """
        super().__init__(name=name, tags=tags, number_of_classes=number_of_classes)

    def update(self, batch: dict):
        raise NotImplementedError

    def update_value(
        self,
        values: torch.Tensor,
        reduce_by: Optional[torch.Tensor] = None,
        weight_by: Optional[torch.Tensor] = None,
    ):
        """
        Args:
            values (torch.Tensor): Values of the metric
            reduce_by (Optional[torch.Tensor], optional): A single or per example divisor of the values. Defaults to batch size.
            weight_by (Optional[torch.Tensor], optional): A single or per example weights for the running mean. Defaults to `reduce_by`.
        """
        values = detach(values)
        reduce_by = detach(reduce_by)

        numel = values.numel() if isinstance(values, torch.Tensor) else 1
        value = values.sum().tolist() if isinstance(values, torch.Tensor) else values

        reduce_by = (
            reduce_by.sum().tolist()
            if isinstance(reduce_by, torch.Tensor)
            else (reduce_by or numel)
        )

        weight_by = (
            weight_by.sum().tolist()
            if isinstance(weight_by, torch.Tensor)
            else (weight_by or reduce_by)
        )

        values = value / reduce_by

        d = self.weight_by + weight_by
        w1 = self.weight_by / d
        w2 = weight_by / d

        self._values = (
            self._values * w1 + values * w2
        )  # Reduce between batches (over entire epoch)

        self.weight_by = d

    def compute(self) -> torch.Tensor:
        return self._values

    def reset(self):
        self._values = torch.tensor(0.0).to(self.device)
        self.weight_by = torch.tensor(0.0).to(self.device)


class LossMetric(RunningMeanMetric):
    base_tags = {"losses"}
    higher_is_better = False

    def __init__(
        self,
        name: str = "loss",
        tags: set[str] = None,
        number_of_classes: Optional[int] = None,
        filter_codes: bool = False,
    ):
        super().__init__(
            name=name,
            tags=tags,
            number_of_classes=number_of_classes,
        )
        self.filter_codes = filter_codes

    def update(self, batch: dict[str, torch.Tensor]):
        loss = detach(batch["loss"]).cpu()
        self.update_value(loss, reduce_by=loss.numel(), weight_by=loss.numel())

In [ ]:
def get_metric_collection(
    config: OmegaConf,
    number_of_classes: int,
    code_system_code_indices: Optional[torch.Tensor] = None,
    split_code_indices: Optional[torch.Tensor] = None,
    code_system_name: Optional[str] = None,
) :
    metric_list = []
    for metric in config:
        metric_class = metric['name']
        metric_list.append(
            globals()[metric_class](number_of_classes=number_of_classes, **metric['configs'])
        )

    if code_system_code_indices is not None and split_code_indices is not None:
        # Get overlapping indices
        code_indices = torch.tensor(
            np.intersect1d(code_system_code_indices, split_code_indices)
        )
    elif code_system_code_indices is not None:
        code_indices = code_system_code_indices.clone()
    elif split_code_indices is not None:
        code_indices = split_code_indices.clone()
    else:
        code_indices = None

    return MetricCollection(
        metrics=metric_list,
        code_indices=code_indices,
        code_system_name=code_system_name,
    )
def get_metric_collections(
    config: OmegaConf,
    number_of_classes: int,
    split_names: list[str] = ["train", "train_val", "val", "test"],
    splits_with_multiple_code_systems: set[str] = {"train_val", "val", "test"},
    code_system2code_indices: dict[str, torch.Tensor] = None,
    split2code_indices: dict[str, torch.Tensor] = None,
) :
    metric_collections = defaultdict(dict)
    for split_name in split_names:
        if split2code_indices is not None:
            split_code_indices = split2code_indices.get(split_name)
        else:
            split_code_indices = None

        metric_collections[split_name]["all"] = get_metric_collection(
            config=config,
            number_of_classes=number_of_classes,
            split_code_indices=split_code_indices,
        )

        if split_name not in splits_with_multiple_code_systems:
            continue

        if code_system2code_indices is None:
            continue

        for (
            code_system_name,
            code_system_code_indices,
        ) in code_system2code_indices.items():
            metric_collections[split_name][code_system_name] = get_metric_collection(
                config=config,
                number_of_classes=number_of_classes,
                code_system_code_indices=code_system_code_indices,
                split_code_indices=split_code_indices,
                code_system_name=code_system_name,
            )

    return metric_collections

## Evaluation Metrics Mentioned in Paper

In [ ]:
metrics_config = [
    {"name": "F1Score", "configs": {"average": "micro", "threshold": 0.5}},
    {"name": "Precision", "configs": {"average": "micro", "threshold": 0.5}},
    {"name": "AUC", "configs": {"average": "micro"}},
    {"name": "Precision_K", "configs": {"k": 1}},
    {"name": "Precision_K", "configs": {"k": 5}},
    {"name": "Precision_K", "configs": {"k": 8}},
    {"name": "Precision_K", "configs": {"k": 15}},

]


In [ ]:
metric_collections = get_metric_collections(
        config=metrics_config,
        number_of_classes=lookups.data_info["num_classes"],
        code_system2code_indices=lookups.code_system2code_indices,
        split2code_indices=lookups.split2code_indices,
    )

## Dataset Utils

In [ ]:
from torch.utils.data import Dataset
class HuggingfaceDataset(Dataset):
    def __init__(
        self,
        data: list[pa.RecordBatch],
        text_transform: Callable,
        label_transform: Callable,
        lookups: Lookups,
        chunk_size: int = 128,
        split_name: str = "train",
    ) -> None:
        super().__init__()
        self.data = data
        self.split_name = split_name
        self.text_transform = text_transform
        self.label_transform = label_transform
        self.chunk_size = chunk_size
        self.lookups = lookups

    def __len__(self):
        return len(self.data)

    def __getitem__(
        self, idx: int
    ) -> tuple[torch.Tensor, torch.Tensor, int, int, torch.Tensor]:
        token_ids, targets, id, num_tokens, attention_mask = self.data[idx]
        targets = self.label_transform(targets)
        return token_ids, targets, id, num_tokens, attention_mask

    def collate_fn(self, batch: tuple[list, list, list, list]) -> Batch:
        token_ids, targets, ids, num_tokens, attention_mask = zip(*batch)
        data = self.text_transform.seq2batch(token_ids, chunk_size=self.chunk_size)
        attention_mask = self.text_transform.seq2batch(
            attention_mask, chunk_size=self.chunk_size
        )
        targets = self.label_transform.seq2batch(targets)
        ids = torch.tensor(ids)
        num_tokens = torch.tensor(num_tokens)
        return Batch(
            data=data,
            targets=targets,
            ids=ids,
            num_tokens=num_tokens,
            attention_mask=attention_mask,
        )

In [ ]:
def get_datasets(
    data: Data,
    text_transform,
    label_transform,
    lookups,
) :

    datasets_dict = {}

    train_data = data.train
    val_data = data.val
    test_data = data.test
    del data.df

    datasets_dict["train"] = HuggingfaceDataset(
        train_data,
        split_name="train",
        text_transform=text_transform,
        label_transform=label_transform,
        lookups=lookups,
        chunk_size = 128,
    )
    datasets_dict["val"] = HuggingfaceDataset(
        val_data,
        split_name="val",
        text_transform=text_transform,
        label_transform=label_transform,
        lookups=lookups,
        chunk_size = 128,
    )
    datasets_dict["test"] = HuggingfaceDataset(
        test_data,
        split_name="test",
        text_transform=text_transform,
        label_transform=label_transform,
        lookups=lookups,
        chunk_size = 128,
    )
    return datasets_dict

## Get the datasets

In [ ]:
datasets = get_datasets(
        data=data,
        text_transform=text_transform,
        label_transform=label_transform,
        lookups=lookups,
    )

## Dataloader utils

In [ ]:
from torch.utils.data import Sampler
class BySequenceLengthSampler(Sampler):
    def __init__(
        self,
        dataset,
        bucket_boundaries: list[int],
        batch_size: int = 64,
        drop_last: bool = True,
    ):
        data = dataset.data
        self.data_len = len(dataset)
        ind_n_len = []
        for example_index, (token_ids, _, _, _, _) in enumerate(data):
            num_tokens = token_ids.size(0)
            ind_n_len.append(
                (example_index, num_tokens)
            )  # 3rd index of the tuple is the length of the sequence

        self.ind_n_len = ind_n_len
        self.bucket_boundaries = bucket_boundaries
        self.batch_size = batch_size
        self.drop_last = drop_last

        if self.drop_last:
            print(
                "WARNING: drop_last=True, dropping last non batch-size batch in every bucket ... "
            )

        self.boundaries = list(self.bucket_boundaries)
        self.buckets_min = torch.tensor([np.iinfo(np.int32).min] + self.boundaries)
        self.buckets_max = torch.tensor(self.boundaries + [np.iinfo(np.int32).max])
        self.boundaries = torch.tensor(self.boundaries)

    def shuffle_tensor(self, t):
        return t[torch.randperm(len(t))]

    def __iter__(self):
        data_buckets = dict()
        # where p is the id number and seq_len is the length of this id number.
        for p, seq_len in self.ind_n_len:
            pid = self.element_to_bucket_id(p, seq_len)
            if pid in data_buckets.keys():
                data_buckets[pid].append(p)
            else:
                data_buckets[pid] = [p]

        for k in data_buckets.keys():
            data_buckets[k] = torch.tensor(data_buckets[k])

        iter_list = []
        for k in data_buckets.keys():
            t = self.shuffle_tensor(data_buckets[k])
            batch = torch.split(t, self.batch_size, dim=0)

            if self.drop_last and len(batch[-1]) != self.batch_size:
                batch = batch[:-1]

            iter_list += batch

        random.shuffle(
            iter_list
        )  # shuffle all the batches so they arent ordered by bucket
        # size
        for batch in iter_list:
            yield batch.numpy().tolist()  # as it was stored in an array

    def __len__(self):
        return (self.data_len // self.batch_size) + 1 - int(self.drop_last)

    def element_to_bucket_id(self, x, seq_length):
        valid_buckets = (seq_length >= self.buckets_min) * (
            seq_length < self.buckets_max
        )
        bucket_id = valid_buckets.nonzero()[0].item()

        return bucket_id

In [ ]:
from torch.utils.data import DataLoader
def get_dataloaders(datasets_dict, batch_size = batch_size, max_batch_size = max_batch_size):
    dataloaders = {}
    train_batch_size = min(batch_size, max_batch_size)
    pprint(f"Train batch size: {train_batch_size}")

    batch_sampler = BySequenceLengthSampler(
        dataset=datasets_dict["train"],
        batch_size=train_batch_size,
        drop_last=True,
        bucket_boundaries = [
    400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2600, 3000, 4000
]
,
    )
    dataloaders["train"] = DataLoader(
        datasets_dict["train"],
        batch_sampler=batch_sampler,
        collate_fn=datasets_dict["train"].collate_fn,
        num_workers=0,
        pin_memory=False,
    )

    dataloaders["train_val"] = DataLoader(
        datasets_dict["train"],
        batch_size=max_batch_size,
        shuffle=False,
        collate_fn=datasets_dict["val"].collate_fn,
        num_workers=0,
        pin_memory=False,
    )

    dataloaders["val"] = DataLoader(
        datasets_dict["val"],
        batch_size=max_batch_size,
        shuffle=False,
        collate_fn=datasets_dict["val"].collate_fn,
        num_workers=0,
        pin_memory=False,
    )

    dataloaders["test"] = DataLoader(
        datasets_dict["test"],
        batch_size=max_batch_size,
        shuffle=False,
        collate_fn=datasets_dict["test"].collate_fn,
        num_workers=0,
        pin_memory=False,
    )

    return dataloaders

## Get Dataloaders

In [ ]:
dataloaders = get_dataloaders(datasets_dict=datasets)

## Define Optimizer for Model Training

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.0001, weight_decay = 0.01)

In [ ]:
accumulate_grad_batches = int(max(max_batch_size/batch_size, 1))

## Calculate Number of Training Steps

In [ ]:
num_training_steps = (
        math.ceil(len(dataloaders["train"]) / accumulate_grad_batches)
        * epochs
    )

## Define Learning Rate Scheduler

In [ ]:
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer = optimizer,
                                                          mode = 'max',
                                                          min_lr=0.0001,
                                                          factor=0.9,
                                                          patience=5)

## Callback Utils

In [ ]:
class BaseCallback:
    def __init__(self):
        pass

    def on_initialisation_end(self, trainer=None):
        pass

    def on_train_begin(self, trainer=None):
        pass

    def on_train_end(self, trainer=None):
        pass

    def on_val_begin(self, trainer=None):
        pass

    def on_val_end(self, trainer=None):
        pass

    def on_epoch_begin(self, trainer=None):
        pass

    def on_epoch_end(self, trainer=None):
        pass

    def on_batch_begin(self, trainer=None):
        pass

    def on_batch_end(self, trainer=None):
        pass

    def on_fit_begin(self, trainer=None):
        pass

    def on_fit_end(self, trainer=None):
        pass

    def log_dict(
        self, nested_dict: dict[str, dict[str, torch.Tensor]], epoch: int
    ) -> None:
        pass

    def on_end(self, trainer=None):
        pass
class SaveBestModelCallback(BaseCallback):
    def __init__(self):
        super().__init__()
        self.prev_best = None
        self.split_name = "val"
        self.target_name = "all"
        self.metric_name = "precision@8"

    def on_epoch_end(self, trainer=None):
        best_metric = trainer.metric_collections[self.split_name][
            self.target_name
        ].get_best_metric(self.metric_name)
        if self.prev_best is None or best_metric != self.prev_best:
            self.prev_best = best_metric
            trainer.save_checkpoint("best_model.pt")
            print("Saved best model")

    def on_fit_end(self, trainer=None):
        trainer.load_checkpoint("best_model.pt")
        print("Loaded best model")

class EarlyStoppingCallback(BaseCallback):
    def __init__(self):
        super().__init__()
        self.split_name = "val"
        self.target_name = "all"
        self.metric_name = "precision@8"
        self.patience = 10
        self.counter = 0
        self.prev_best = None

    def on_epoch_end(self, trainer=None):
        """On the end of each epoch, test if the validation metric has improved. If it hasn't improved for self.patience epochs, stop training.

        Args:
            trainer (Trainer, optional): Trainer class. Defaults to None.
        """
        best_metric = trainer.metric_collections[self.split_name][
            self.target_name
        ].get_best_metric(self.metric_name)
        if self.prev_best is None or best_metric > self.prev_best:
            self.prev_best = best_metric
            self.counter = 0
        else:
            self.counter += 1
        if self.counter >= self.patience:
            trainer.stop_training = True
            print(
                f"Early stopping: {self.counter} epochs without improvement for {self.metric_name}"
            )

In [ ]:
def get_callbacks():
    callbacks_list = []
    callbacks_list.append(SaveBestModelCallback())
    callbacks_list.append(EarlyStoppingCallback())
    return callbacks_list

## Get the Callbacks

In [ ]:
callbacks = get_callbacks()

## Trainer Utils

In [ ]:
class Trainer:
    def __init__(
        self,
        epochs,
        data,
        model,
        optimizer,
        dataloaders,
        metric_collections,
        callbacks,
        lr_scheduler = None,
        lookups = None,
        accumulate_grad_batches = 1,
    ) :
        self.data = data
        self.model = model
        self.optimizer = optimizer
        self.dataloaders = dataloaders
        self.callbacks = callbacks
        self.device = "cpu"
        self.metric_collections = metric_collections
        self.lr_scheduler = lr_scheduler
        self.lookups = lookups
        self.accumulate_grad_batches = accumulate_grad_batches
        pprint(f"Accumulating gradients over {self.accumulate_grad_batches} batch(es).")
        self.validate_on_training_data = True
        self.print_metrics = True
        self.epochs = epochs
        self.epoch = 0
        self.use_amp = True
        self.threshold_tuning = False
        self.gradient_scaler = torch.cuda.amp.GradScaler(enabled=self.use_amp)
        self.experiment_path = Path(mkdtemp())
        self.current_val_results = None
        self.stop_training = False
        self.best_db = 0.5
        self.val_f1_micro_list = []
        self.val_precision_micro_list = []
        self.val_precision_1_list = []
        self.val_precision_5_list = []
        self.val_precision_8_list = []
        self.val_precision_15_list = []
        self.train_f1_micro_list = []
        self.train_precision_micro_list = []
        self.train_precision_1_list = []
        self.train_precision_5_list = []
        self.train_precision_8_list = []
        self.train_precision_15_list = []
        self.on_initialisation_end()

    def fit(self) -> None:
        """Train and validate the model."""
        try:
            self.on_fit_begin()

            for _ in range(self.epoch, self.epochs):
                if self.stop_training:
                    break
                self.on_epoch_begin()
                self.train_one_epoch(self.epoch)
                if self.validate_on_training_data:
                    self.train_val(self.epoch, "train_val")
                self.val(self.epoch, "val")
                self.on_epoch_end()
                self.epoch += 1
            self.on_fit_end()
            self.val(self.epoch, "val", evaluating_best_model=True)
            self.val(self.epoch, "test", evaluating_best_model=True)
            self.save_final_model()
        except KeyboardInterrupt:
            pprint("Training interrupted by user. Stopping training")
        del self.model
        gc.collect()
        torch.cuda.empty_cache()
        self.on_end()

    def train_one_epoch(self, epoch: int) -> None:
        """Train the model for one epoch.

        Args:
            epoch (int): The current epoch.
        """
        self.model.train()
        self.on_train_begin()
        num_batches = len(self.dataloaders["train"])
        for batch_idx, batch in enumerate(
            track(self.dataloaders["train"], description=f"Epoch: {epoch} | Training")
        ):
            batch = batch.to(self.device)
            with torch.autocast(
                device_type="cuda", dtype=torch.float16, enabled=self.use_amp
            ):
                output = self.model.training_step(batch)
                loss = output["loss"] / self.accumulate_grad_batches
            self.gradient_scaler.scale(loss).backward()
            if ((batch_idx + 1) % self.accumulate_grad_batches == 0) or (
                batch_idx + 1 == num_batches
            ):
                self.gradient_scaler.step(self.optimizer)
                self.gradient_scaler.update()
                if self.lr_scheduler is not None:
                    if not isinstance(
                        self.lr_scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau
                    ):
                        self.lr_scheduler.step()
                self.optimizer.zero_grad()
            self.update_metrics(output, "train")
        self.on_train_end(epoch)

    def train_val(self, epoch, split_name: str = "train_val") -> None:
        """Validate on the training data. This is useful for testing for overfitting. Due to memory constraints, we donøt save the outputs.

        Args:
            epoch (_type_): _description_
            split_name (str, optional): _description_. Defaults to "train_val".
        """
        self.model.eval()
        self.on_val_begin()
        with torch.no_grad():
            for batch in track(
                self.dataloaders[split_name],
                description=f"Epoch: {epoch} | Validating on training data",
            ):
                with torch.autocast(
                    device_type="cuda", dtype=torch.float16, enabled=self.use_amp
                ):
                    output = self.model.validation_step(batch.to(self.device))
                self.update_metrics(output, split_name)
            self.on_val_end(split_name, epoch)

    def val(
        self, epoch, split_name: str = "val", evaluating_best_model: bool = False
    ) -> None:
        self.model.eval()
        self.on_val_begin()
        logits = []
        targets = []
        logits_cpu = []
        targets_cpu = []
        ids = []
        with torch.no_grad():
            for idx, batch in enumerate(
                track(
                    self.dataloaders[split_name],
                    description=f"Epoch: {epoch} | Validating on {split_name}",
                )
            ):
                with torch.autocast(
                    device_type="cuda", dtype=torch.float16, enabled=self.use_amp
                ):
                    output = self.model.validation_step(batch.to(self.device))
                self.update_metrics(output, split_name)
                logits.append(output["logits"])
                targets.append(output["targets"])
                ids.append(batch.ids)
                if idx % 1000 == 0:
                    # move to cpu to save gpu memory
                    logits_cpu.append(torch.cat(logits, dim=0).cpu())
                    targets_cpu.append(torch.cat(targets, dim=0).cpu())
                    logits = []
                    targets = []
            logits_cpu.append(torch.cat(logits, dim=0).cpu())
            targets_cpu.append(torch.cat(targets, dim=0).cpu())

            logits = torch.cat(logits_cpu, dim=0)
            targets = torch.cat(targets_cpu, dim=0)
            ids = torch.cat(ids, dim=0)
        self.on_val_end(split_name, epoch, logits, targets, ids, evaluating_best_model)

    def update_metrics(self, outputs: dict[str, torch.Tensor], split_name: str) -> None:
        for target_name in self.metric_collections[split_name].keys():
            self.metric_collections[split_name][target_name].update(outputs)

    def calculate_metrics(
        self,
        split_name: str,
        logits: Optional[torch.Tensor] = None,
        targets: Optional[torch.Tensor] = None,
        evaluating_best_model: bool = False,
    ) -> dict[str, dict[str, torch.Tensor]]:
        results_dict = defaultdict(dict)
        if split_name == "val":
            for target_name in self.metric_collections[split_name].keys():
                results_dict[split_name][target_name] = self.metric_collections[
                    split_name
                ][target_name].compute()
        else:
            for target_name in self.metric_collections[split_name].keys():
                results_dict[split_name][target_name] = self.metric_collections[
                    split_name
                ][target_name].compute(logits, targets)

        if self.threshold_tuning and split_name == "val":
            best_result, best_db = f1_score_db_tuning(logits, targets)
            results_dict[split_name]["all"] |= {"f1_micro_tuned": best_result}
            if evaluating_best_model:
                pprint(f"Best threshold: {best_db}")
                pprint(f"Best result: {best_result}")
                for target_name in self.metric_collections["test"]:
                    self.metric_collections["test"][target_name].set_threshold(best_db)
            self.best_db = best_db
        return results_dict

    def reset_metric(self, split_name: str) -> None:
        for target_name in self.metric_collections[split_name].keys():
            self.metric_collections[split_name][target_name].reset_metrics()

    def reset_metrics(self) -> None:
        for split_name in self.metric_collections.keys():
            for target_name in self.metric_collections[split_name].keys():
                self.metric_collections[split_name][target_name].reset_metrics()

    def on_initialisation_end(self) -> None:
        for callback in self.callbacks:
            callback.on_initialisation_end(self)

    def on_fit_begin(self) -> None:
        for callback in self.callbacks:
            callback.on_fit_begin(self)

    def on_fit_end(self) -> None:
        for callback in self.callbacks:
            callback.on_fit_end(self)

    def on_train_begin(self) -> None:
        for callback in self.callbacks:
            callback.on_train_begin()

    def on_train_end(self, epoch: int) -> None:
        results_dict = self.calculate_metrics(split_name="train")
        self.current_train_results = results_dict
        results_dict["lr"] = self.optimizer.param_groups[0]["lr"]
        self.log_dict(results_dict, epoch)
        for callback in self.callbacks:
            callback.on_train_end()

    def on_val_begin(self) -> None:
        for callback in self.callbacks:
            callback.on_val_begin()

    def on_val_end(
        self,
        split_name: str,
        epoch: int,
        logits: torch.Tensor = None,
        targets: torch.Tensor = None,
        ids: torch.Tensor = None,
        evaluating_best_model: bool = False,
    ) -> None:
        results_dict = self.calculate_metrics(
            split_name=split_name,
            logits=logits,
            targets=targets,
            evaluating_best_model=evaluating_best_model,
        )
        self.current_val_results = results_dict
        self.log_dict(results_dict, epoch)
        for callback in self.callbacks:
            callback.on_val_end()

        if evaluating_best_model:
            self.save_predictions(
                split_name=split_name, logits=logits, targets=targets, ids=ids
            )

    def save_predictions(
        self,
        split_name: str = "test",
        logits: torch.Tensor = None,
        targets: torch.Tensor = None,
        ids: torch.Tensor = None,
    ):
        from time import time

        tic = time()
        pprint("Saving predictions")
        label_transform = self.dataloaders[split_name].dataset.label_transform
        code_names = label_transform.get_classes()
        logits = logits.numpy()
        pprint("Building dataframe")
        df = pd.DataFrame(logits, columns=code_names)
        pprint("Adding targets")
        df[TARGET_COLUMN] = list(map(label_transform.inverse_transform, targets))
        pprint("Adding ids")
        df[ID_COLUMN] = ids.numpy()
        pprint("Saving dataframe")
        df.to_feather(self.experiment_path / f"predictions_{split_name}.feather")
        pprint("Saved predictions in {:.2f} seconds".format(time() - tic))

    def on_epoch_begin(self) -> None:
        self.reset_metrics()
        for callback in self.callbacks:
            callback.on_epoch_begin(self)

    def on_epoch_end(self) -> None:
        self.val_f1_micro_list.append((self.epoch, self.current_val_results['val']['all']['f1_micro']))
        self.val_precision_micro_list.append((self.epoch, self.current_val_results['val']['all']['precision_micro']))
        self.val_precision_1_list.append((self.epoch, self.current_val_results['val']['all']['precision@1']))
        self.val_precision_5_list.append((self.epoch, self.current_val_results['val']['all']['precision@5']))
        self.val_precision_8_list.append((self.epoch, self.current_val_results['val']['all']['precision@8']))
        self.val_precision_15_list.append((self.epoch, self.current_val_results['val']['all']['precision@15']))
        self.train_f1_micro_list.append((self.epoch, self.current_train_results['train']['all']['f1_micro']))
        self.train_precision_micro_list.append((self.epoch, self.current_train_results['train']['all']['precision_micro']))
        self.train_precision_1_list.append((self.epoch, self.current_train_results['train']['all']['precision@1']))
        self.train_precision_5_list.append((self.epoch, self.current_train_results['train']['all']['precision@5']))
        self.train_precision_8_list.append((self.epoch, self.current_train_results['train']['all']['precision@8']))
        self.train_precision_15_list.append((self.epoch, self.current_train_results['train']['all']['precision@15']))
        if self.lr_scheduler is not None:
            if isinstance(
                self.lr_scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau
            ):
                self.lr_scheduler.step(
                    self.current_val_results["val"]["all"]["f1_micro"]
                )

        for callback in self.callbacks:
            callback.on_epoch_end(self)

    def on_batch_begin(self) -> None:
        for callback in self.callbacks:
            callback.on_batch_begin()

    def on_batch_end(self) -> None:
        for callback in self.callbacks:
            callback.on_batch_end()

    def log_dict(
        self, nested_dict: dict[str, dict[str, torch.Tensor]], epoch: int
    ) -> None:
        if self.print_metrics:
            self.print(nested_dict)
        for callback in self.callbacks:
            callback.log_dict(nested_dict, epoch)

    def on_end(self) -> None:
        for callback in self.callbacks:
            callback.on_end()

    def print(self, nested_dict: dict[str, dict[str, torch.Tensor]]) -> None:
        for split_name in nested_dict.keys():
            pprint(nested_dict[split_name])

    def to(self, device: str) -> "Trainer":
        self.model.to(device)
        for split_name in self.metric_collections.keys():
            for target_name in self.metric_collections[split_name].keys():
                self.metric_collections[split_name][target_name].to(device)
        self.device = device
        return self

    def save_checkpoint(self, file_name: str) -> None:
        checkpoint = {
            "model": self.model.state_dict(),
            "optimizer": self.optimizer.state_dict(),
            "scaler": self.gradient_scaler.state_dict(),
            "epoch": self.epoch,
            "db": self.best_db,
        }
        torch.save(checkpoint, self.experiment_path / file_name)
        pprint("Saved checkpoint to {}".format(self.experiment_path / file_name))

    def load_checkpoint(self, file_name: str) -> None:
        checkpoint = torch.load(self.experiment_path / file_name)
        self.model.load_state_dict(checkpoint["model"])
        self.optimizer.load_state_dict(checkpoint["optimizer"])
        self.gradient_scaler.load_state_dict(checkpoint["scaler"])
        self.epoch = checkpoint["epoch"]
        self.best_db = checkpoint["db"]
        pprint("Loaded checkpoint from {}".format(self.experiment_path / file_name))

    def plot_metrics(self):
        epochs, val_f1_micro = zip(*self.val_f1_micro_list)
        _, val_precision_micro = zip(*self.val_precision_micro_list)
        _, val_precision_1 = zip(*self.val_precision_1_list)
        _, val_precision_5 = zip(*self.val_precision_5_list)
        _, val_precision_8 = zip(*self.val_precision_8_list)
        _, val_precision_15 = zip(*self.val_precision_15_list)

        _, train_f1_micro = zip(*self.train_f1_micro_list)
        _, train_precision_micro = zip(*self.train_precision_micro_list)
        _, train_precision_1 = zip(*self.train_precision_1_list)
        _, train_precision_5 = zip(*self.train_precision_5_list)
        _, train_precision_8 = zip(*self.train_precision_8_list)
        _, train_precision_15 = zip(*self.train_precision_15_list)

        # Plotting Training Metrics
        plt.figure(figsize=(10, 6))  # Set figure size
        plt.plot(epochs, train_f1_micro, label='F1 Micro', marker='o', color='royalblue', linestyle='-')
        plt.plot(epochs, train_precision_micro, label='Precision Micro', marker='s', color='darkorange', linestyle='--')
        plt.plot(epochs, train_precision_1, label='Precision 1', marker='^', color='forestgreen', linestyle='-.')
        plt.plot(epochs, train_precision_5, label='Precision 5', marker='D', color='firebrick', linestyle=':')
        plt.plot(epochs, train_precision_8, label='Precision 8', marker='P', color='mediumslateblue', linestyle='-')
        plt.plot(epochs, train_precision_15, label='Precision 15', marker='X', color='darkgoldenrod', linestyle='--')
        plt.xlabel('Epoch', fontsize=14)
        plt.ylabel('Value', fontsize=14)
        plt.title('Training Metrics vs. Epoch', fontsize=16)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.legend(fontsize=12)  # Add legend with increased font size
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()


        # Plotting Validation Metrics
        plt.figure(figsize=(10, 6))  # Set figure size
        plt.plot(epochs, val_f1_micro, label='F1 Micro', marker='o', color='royalblue', linestyle='-')
        plt.plot(epochs, val_precision_micro, label='Precision Micro', marker='s', color='darkorange', linestyle='--')
        plt.plot(epochs, val_precision_1, label='Precision 1', marker='^', color='forestgreen', linestyle='-.')
        plt.plot(epochs, val_precision_5, label='Precision 5', marker='D', color='firebrick', linestyle=':')
        plt.plot(epochs, val_precision_8, label='Precision 8', marker='P', color='mediumslateblue', linestyle='-')
        plt.plot(epochs, val_precision_15, label='Precision 15', marker='X', color='darkgoldenrod', linestyle='--')
        plt.xlabel('Epoch', fontsize=14)
        plt.ylabel('Value', fontsize=14)
        plt.title('Validation Metrics vs. Epoch', fontsize=16)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.legend(fontsize=12)  # Add legend with increased font size
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

    def save_transforms(self) -> None:
        """Save text tokenizer and label encoder"""
        self.dataloaders["train"].dataset.text_transform.save(self.experiment_path)
        self.dataloaders["train"].dataset.label_transform.save(self.experiment_path)

    def save_final_model(self) -> None:
        self.save_checkpoint("final_model.pt")
        self.save_transforms()

## Get the Trainer

In [ ]:
trainer = Trainer(
        epochs = epochs,
        data=data,
        model=model,
        optimizer=optimizer,
        dataloaders=dataloaders,
        metric_collections=metric_collections,
        callbacks=callbacks,
        lr_scheduler=lr_scheduler,
        lookups=lookups,
        accumulate_grad_batches=accumulate_grad_batches,
    ).to(device)

## Set Experiment Path

In [ ]:
trainer.experiment_path = pretrained_model_path

## Start Training

In [ ]:
torch.cuda.empty_cache()
with torch.autocast("cuda"):
    trainer.fit()

## Visualize the Predictions on Validation Set

In [ ]:
val_preds = pd.read_feather("/kaggle/working/RoBERTa-base-PM-M3-Voc-hf/predictions_val.feather")
val_preds

## Visualize Predictions on Test Set

In [ ]:
val_preds = pd.read_feather("/kaggle/working/RoBERTa-base-PM-M3-Voc-hf/predictions_test.feather")
val_preds

###